In [5]:
import os
from pytorch_lightning.utilities.seed import seed_everything

from torch.utils.data.dataloader import T
import numpy as np
# TODO fix mkl problem 
# os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import pywick 
from pywick.models.segmentation import deeplab_v3_plus
from torch.utils.data.sampler import SubsetRandomSampler
from utils.util import json_file_to_dict_args, json_file_to_pyobj,get_tags
from dataio.loader import get_dataset, get_dataset_path
from dataio.transformation import get_dataset_transformation
from torch.utils.data import DataLoader, dataset
from pywick.modules import ModuleTrainer
import pywick.metrics as pwm
from utils.error_logger import ErrorLogger
import torch
import numpy
from tqdm import tqdm
import json
from utils.visualiser import Visualiser
from utils.error_logger import ErrorLogger
import pytorch_lightning as pl
from models import get_model


In [6]:
from torchvision import transforms
from torch.utils.data import DataLoader
import numpy as np
import PIL.Image
import matplotlib.pyplot as plt
class DivideIntoPatches:
    def __init__(self, patch_size):
        self.patch_size = patch_size

    def __call__(self, img):
        height, width = img.shape[-2:]
        patches = []
        for i in range(0, height - self.patch_size + 1, self.patch_size):
            for j in range(0, width - self.patch_size + 1, self.patch_size):
                patches.append(img[ :, i:i + self.patch_size, j:j + self.patch_size])
        return torch.stack(patches, dim=0)
class ResizePatches:
    def __init__(self,pz):
        self.patch_size = pz
    def __call__(self, img):
        imgs = img.reshape(-1,img.shape[-3],img.shape[-2],img.shape[-1])
        return torch.nn.functional.interpolate(imgs, size=(self.patch_size,self.patch_size), mode='bilinear', align_corners=False)
def get_data(pz = 80, img_size=224, bs=None,split="train",pz_=None):
    patch_size = pz
    img_size = img_size
    num_patches = (img_size // patch_size) ** 2
    bs = 90 // num_patches if bs is None else bs
    if num_patches <5:
        bs=bs//8
    bs = 1 if bs == 0 else bs
    print("config:\n", "patch_size:", patch_size, "img_size:", img_size, "num_patches:", num_patches, "bs:", bs)
    if pz_ is None:
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((img_size, img_size)),
            transforms.ConvertImageDtype(torch.float),
            DivideIntoPatches(patch_size=patch_size), # takes an image tensor and returns a list of patches stacked as (H // patch_size **2 x H x W x C)
        ])
    else:
        transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((img_size, img_size)),
        transforms.ConvertImageDtype(torch.float),
        DivideIntoPatches(patch_size=patch_size), # takes an image tensor and returns a list of patches stacked as (H // patch_size **2 x H x W x C)
        ResizePatches(pz=pz_)
        ])

    # data = wss_dataset_class("/home/uz1/data/wsss/train/1.training", 'all',
                            #  transform)
    # data = HDF5Dataset("/home/uz1/DATA!/pcam/pcam/training_split.h5","/home/uz1/DATA!/pcam/Labels/Labels/camelyonpatch_level_2_split_train_y.h5",transform=transform)
    from medmnist.dataset import PathMNIST, BreastMNIST,OCTMNIST,ChestMNIST,PneumoniaMNIST,DermaMNIST,RetinaMNIST,BloodMNIST,TissueMNIST,OrganAMNIST,OrganCMNIST,OrganSMNIST
    # using a unified ataset of medmnist
    data = PathMNIST(root='/home/uz1/DATA!/medmnist', split=split,transform=transform)
    loader = DataLoader(data, batch_size=bs, drop_last=True, num_workers=14)
    return data,loader

In [7]:
from models.networks.swin_transformer import SwinTransformer
from torchvision.models import swin_transformer
from pl_bolts.models.autoencoders.components import (
    resnet18_decoder,
    resnet18_encoder,
    ResNetDecoder,
    DecoderBlock,
)
from torch import nn
import torch.nn.functional as F
class Interpolate(nn.Module):
    """nn.Module wrapper for F.interpolate."""

    def __init__(self, size=None, scale_factor=None):
        super().__init__()
        self.size, self.scale_factor = size, scale_factor

    def forward(self, x):
        return F.interpolate(x, size=self.size, scale_factor=self.scale_factor)
def resize_conv1x1(in_planes, out_planes, scale=1):
    """upsample + 1x1 convolution with padding to avoid checkerboard artifact."""
    if scale == 1:
        return conv1x1(in_planes, out_planes)
    return nn.Sequential(Interpolate(scale_factor=scale), conv1x1(in_planes, out_planes))
def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution."""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

class Resdecoder(nn.Module):
    def __init__(self, x,latent_dim=256, input_height=224, first_conv=False, maxpool1=False):
        # super().__init__(   DecoderBlock, [2, 2, 2, 2],latent_dim=latent_dim, input_height=input_height, first_conv=first_conv, maxpool1=maxpool1)
        super().__init__()
        self.block = DecoderBlock
        self.latent_dim = latent_dim
        self.expansion = self.block.expansion
        self.first_conv = first_conv
        self.maxpool1 = maxpool1
        self.input_height = input_height
        self.layers = [2, 2, 2, 2]
        self.upscale_factor = 8
        

        _, self.c, self.h, self.w = x.shape
        block = self.block
        layers = self.layers
        out_put_shape = self.c # * self.h * self.w
        self.linear = torch.nn.Linear(self.latent_dim, out_put_shape * self.h * self.w)
        self.inplanes = out_put_shape



        self.layer1 = self._make_layer(block, 256, layers[0], scale=2)
        self.layer2 = self._make_layer(block, 128, layers[1], scale=2)
        self.layer3 = self._make_layer(block, 64, layers[2], scale=2)

        if self.maxpool1:
            self.layer4 = self._make_layer(block, 64, layers[3], scale=2)
            self.upscale_factor *= 2
        else:
            self.layer4 = self._make_layer(block, 64, layers[3])

        if self.first_conv:
            self.upscale = Interpolate(scale_factor=2)
            self.upscale_factor *= 2
        else:
            self.upscale = Interpolate(scale_factor=1)

        # interpolate after linear layer using scale factor
        self.upscale1 = Interpolate(size=self.input_height // self.upscale_factor)

        self.conv1 = nn.Conv2d(64 * block.expansion, 3, kernel_size=3, stride=1, padding=1, bias=False)
    def _make_layer(self, block, planes, blocks, scale=1):
        upsample = None
        if scale != 1 or self.inplanes != planes * block.expansion:
            upsample = nn.Sequential(
                resize_conv1x1(self.inplanes, planes * block.expansion, scale),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, scale, upsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.linear(x)

        x = x.view(x.size(0), self.c, self.h, self.w)
        x = self.upscale1(x)
        # print('in to l1',x.shape)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.upscale(x)

        x = self.conv1(x)

        return x

/home/uz1/miniconda3/envs/ML/lib/python3.7/site-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:35: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
/home/uz1/miniconda3/envs/ML/lib/python3.7/site-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:93: UnderReviewWarning: The feature FeatureMapContrastiveTask is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  contrastive_task: Union[FeatureM

In [8]:

class VAE(pl.LightningModule):
    def __init__(self, enc_out_dim=512, latent_dim=256, input_height=32, window_size=4,patch_size = 4):
        super().__init__()

        self.save_hyperparameters()
        self.input_height = input_height
        print("using input height ",input_height," and latent dim ",latent_dim," and enc_out_dim ",enc_out_dim)
        # self.encoder = SwinTransformerAE(img_size=input_height,patch_size = patch_size,window_size= window_size,enc_out = enc_out_dim)
        self.encoder = swin_transformer._swin_transformer(
                        patch_size=[patch_size, patch_size],
                        embed_dim=96,
                        depths=[4,6],
                        num_heads=[12, 24],
                        window_size=[window_size, window_size],
                        stochastic_depth_prob=0.1,
                        weights=None,
                        progress=0,
                        num_classes=enc_out_dim,
                    )

    

        # distribution parameters
        self.fc_mu = nn.Linear(enc_out_dim, latent_dim)
        self.fc_var = nn.Linear(enc_out_dim, latent_dim)

        #simulate a pass through the encoder 
        x = self.encoder.features(torch.rand(1,3,input_height, input_height )).permute(0,3,1,2)
        self.decoder = Resdecoder(x,latent_dim=latent_dim,input_height=input_height,first_conv=False,maxpool1=False)

        # for the gaussian likelihood
        self.log_scale = nn.Parameter(torch.Tensor([0.0]))
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-4)

    def gaussian_likelihood(self, mean, logscale, sample):
        scale = torch.exp(logscale)
        dist = torch.distributions.Normal(mean, scale)
        log_pxz = dist.log_prob(sample)
        return log_pxz.sum(dim=(1, 2, 3))

    def kl_divergence(self, z, mu, std):
        # --------------------------
        # Monte carlo KL divergence
        # --------------------------
        # 1. define the first two probabilities (in this case Normal for both)
        p = torch.distributions.Normal(torch.zeros_like(mu),
                                       torch.ones_like(std))
        q = torch.distributions.Normal(mu, std)

        # 2. get the probabilities from the equation
        log_qzx = q.log_prob(z)
        log_pz = p.log_prob(z)

        # kl
        kl = (log_qzx - log_pz)
        kl = kl.sum(-1)
        return kl

    def training_step(self, batch, batch_idx):
        # print(batch)
        x, _ = batch
        #if channels are less than 3, repeat channels
        # print(x.shape)
        # if x.shape[1] < 3:
        #     x = x.repeat(1, 3, 1, 1)
        # print('x in training step',x.shape)
        if x.dim() >4: # b,16,3,h,w -> b*16,3,h,w
            x = x.view(-1, x.shape[2], x.shape[3], x.shape[4])
        # encode x to get the mu and variance parameters
        x_encoded = self.encoder(x)
        # print('x_encoded in training step',x_encoded.shape)
        mu, log_var = self.fc_mu(x_encoded), self.fc_var(x_encoded)
        # detect nan 
        if torch.isnan(mu).any() or torch.isnan(log_var).any():
            print("NAN in mu or log_var")
            print(mu)
            print(log_var)
            print(x_encoded,x_encoded.shape)
            print(x,x.shape)
            raise ValueError("NAN in mu or log_var")
        # sample z from q
        std = torch.exp(log_var / 2)
        q = torch.distributions.Normal(mu, std)
        z = q.rsample()

        # decoded
        # print('z in training step',z.shape)
        x_hat = self.decoder(z)
        # print('x_hat in training step',x_hat.shape)
        # reconstruction loss
        recon_loss_ = self.gaussian_likelihood(x_hat, self.log_scale, x) # old recon_loss 
        # print(recon_loss.shape)
        recon_loss = torch.nn.MSELoss()(x_hat,x)
        # print(recon_loss.shape)

        # kl
        kl = self.kl_divergence(z, mu, std)

        # elbo
        elbo = (kl - recon_loss_) # with old recon_loss
        # elbo = (kl + recon_loss)
        elbo = elbo.mean()

        self.log_dict({
            'elbo': elbo,
            'kl': kl.mean(),
            'recon_loss_mse': recon_loss.mean(),
            'recon_loss_gl': recon_loss_.mean(),
            'reconstruction': recon_loss.mean(),
            'kl': kl.mean(),
        })

        return elbo
    def forward(self,batch):
        x, _ = batch if len(batch)==2 else batch , 0
        #if channels are less than 3, repeat channels
        # print(x.shape)
        # if x.shape[1] < 3:
        #     x = x.repeat(1, 3, 1, 1)
        # print('x in training step',x.shape)
        if x.dim() >4: # b,16,3,h,w -> b*16,3,h,w
            x = x.view(-1, x.shape[2], x.shape[3], x.shape[4])
        # encode x to get the mu and variance parameters
        x_encoded = self.encoder(x)
        # print('x_encoded in training step',x_encoded.shape)
        mu, log_var = self.fc_mu(x_encoded), self.fc_var(x_encoded)
        # detect nan 
        if torch.isnan(mu).any() or torch.isnan(log_var).any():
            print("NAN in mu or log_var")
            print(mu)
            print(log_var)
            print(x_encoded,x_encoded.shape)
            raise ValueError("NAN in mu or log_var")
        # sample z from q
        std = torch.exp(log_var / 2)
        q = torch.distributions.Normal(mu, std)
        z = q.rsample()

        # decoded
        # print('z in training step',z.shape)
        x_hat = self.decoder(z)
        return x_hat
    def validation_step(self, batch, batch_idx):
        x, _ = batch
        #if channels are less than 3, repeat channels
        # print(x.shape)
        # if x.shape[1] < 3:
        #     x = x.repeat(1, 3, 1, 1)
        # print(x.shape)
        if x.dim() >4: # b,16,3,h,w -> b*16,3,h,w
            x = x.view(-1, x.shape[2], x.shape[3], x.shape[4])
        # encode x to get the mu and variance parameters
        x_encoded = self.encoder(x)
        mu, log_var = self.fc_mu(x_encoded), self.fc_var(x_encoded)

        # sample z from q
        std = torch.exp(log_var / 2)
        q = torch.distributions.Normal(mu, std)
        z = q.rsample()

        # decoded

        x_hat = self.decoder(z)

        # reconstruction loss
        recon_loss_ = self.gaussian_likelihood(x_hat, self.log_scale, x)
        # print(recon_loss.shape)
        recon_loss = torch.nn.MSELoss()(x_hat,x)


        # kl
        kl = self.kl_divergence(z, mu, std)

        # elbo
        elbo = (kl - recon_loss_) # with old recon_loss
        # elbo = (kl + recon_loss)
        elbo = elbo.mean()

        self.log_dict({
            'val_elbo': elbo,
            'val_kl': kl.mean(),
            'val_recon_loss_': recon_loss.mean(),
            'val_recon_loss': recon_loss_.mean(),
            'val_reconstruction': recon_loss.mean(),
            'val_kl': kl.mean(),
        })

        return elbo


data , loader = get_data(pz = 112, img_size=  224)
v = VAE(input_height=data[0][0].shape[-1],window_size =28, patch_size = 4)
v.training_step(data[0],0)

config:
 patch_size: 112 img_size: 224 num_patches: 4 bs: 2
using input height  112  and latent dim  256  and enc_out_dim  512


/home/uz1/miniconda3/envs/ML/lib/python3.7/site-packages/ipykernel_launcher.py:81: UnderReviewWarning: The feature DecoderBlock is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
/home/uz1/miniconda3/envs/ML/lib/python3.7/site-packages/pl_bolts/models/autoencoders/components.py:132: UnderReviewWarning: The feature resize_conv3x3 is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  self.conv1 = resize_conv3x3(inplanes, inplanes)
/home/uz1/miniconda3/envs/ML/lib/python3.7/site-packages/pl_bolts/models/autoencoders/components.py:36: 

tensor(52627.7031, grad_fn=<MeanBackward0>)

In [9]:

#lets dfine an nn.Module to wrap the encoder and decoder
class AE(nn.Module):
    def __init__(self,inp):
        super().__init__()
        self.encoder = SwinTransformerAE(inp,4)
        self.decoder = Resdecoder(latent_dim=256,input_height=inp,first_conv=False,maxpool1=False)
    def init_params(self,x):
        # print('in to encoder ',x.shape)
        x = self.encoder.forward(x)
        # print('out from encoder ',x[-1].shape)
        self.decoder.init_params(x[-1])
    def forward(self,x):
        # print('in to encoder ',x.shape)
        out = self.encoder.forward_features(x)
        # print('out from encoder ',out.shape)
        
        out = self.decoder(out)
        return out
# model = AE(inp=data[0][0].shape[-1])
# model.init_params(data[0][0])
# model(data[0][0]).shape

In [10]:
import pytorch_lightning as pl
class AEpl(pl.LightningModule):
    def __init__(self,inp,lr):
        """This is a class for training an autoencoder with pytorch lightning.

        Args:
            inp (int): number of input features
            lr (float): learning rate
        """
        super().__init__()
        self.save_hyperparameters()
        self.model = AE(inp)
        self.loss = nn.MSELoss()
    def forward(self,x):
        return self.model(x)
    #before train
    def init_params(self,x):
        self.model.init_params(x)
    def to_cuda(self):
        self.model.encoder.cuda()
        self.model.decoder.cuda()
    def training_step(self,batch,batch_idx):
        x,_ = batch
        if len(x.shape) > 4:
            x = x.reshape(-1,x.shape[-3],x.shape[-2],x.shape[-1])
        out = self.forward(x)
        loss = self.loss(x,out)
        self.log('train_loss',loss)
        return loss
    def configure_optimizers(self):
        opt = torch.optim.Adam(self.parameters(), lr=self.hparams.lr)
        return opt




In [11]:
#`` lr logging
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from callbacks import  TestReconCallback_vae,TestReconCallback_ae
import os
import random
import datetime
def get_callbacks():
    callbacks = []
    lr_monitor = LearningRateMonitor(logging_interval="epoch")
    callbacks.append(lr_monitor)

    # save checkpoint on last epoch only
    ckpt = ModelCheckpoint(f"/home/uz1/projects/GCN/logging/swin_vae/{datetime.datetime.now().strftime('%Y_%m_%d')}",
                        monitor="elbo",
                        save_weights_only=True)
    callbacks.append(ckpt)
    return callbacks


In [8]:
#data 
pz=64
data , loader = get_data(pz = pz, img_size=  224, )
#clbs 
# add test for mid-train recon viewing
test = [data[x][0] for x in random.sample(range(len(data)), 1)]

test = torch.stack(test, 0).squeeze()
testRecon = TestReconCallback_vae(test)
callbacks = get_callbacks()
callbacks.append(testRecon)
# train 
# model = AEpl(inp=data[0][0].shape[-1],lr=1e-3)
inp = data[0][0].shape[-1]
model2 = VAE(input_height=inp,window_size =7, patch_size = 4)
model = model2
print("transfering from ",model.hparams.input_height, ' To ', model2.hparams.input_height)
#check if hparams are the same
# param check

from torchinfo import summary
print(summary(model, input_size=(1, data[0][0].shape[1], pz, pz),))
if model.hparams.input_height != model2.hparams.input_height:
    # take our current patch size and resize it to the new input size
    data,loader= get_data(pz = model2.hparams.input_height, img_size= 224,pz_=model.hparams.input_height,split='train')
    #readjust callbacks 
    test = [data[x][0] for x in random.sample(range(len(data)), 1)]
    callbacks = get_callbacks()
    test = torch.stack(test, 0).squeeze()
    testRecon = TestReconCallback_vae(test)
    callbacks.append(testRecon)
    print('new data size',data[0][0].shape)
# model.to_cuda()
_,test_loader = get_data(pz = pz, img_size=  224,split='val',pz_=model.hparams.input_height)
# build trainer
trainer = pl.Trainer(gpus=1, max_epochs=10, callbacks=callbacks )
# train
trainer.fit(model, loader,test_loader)

config:
 patch_size: 64 img_size: 224 num_patches: 9 bs: 10
using input height  64  and latent dim  256  and enc_out_dim  512


/home/uz1/miniconda3/envs/ML/lib/python3.7/site-packages/ipykernel_launcher.py:81: UnderReviewWarning: The feature DecoderBlock is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html


transfering from  64  To  64
Layer (type:depth-idx)                                  Output Shape              Param #
VAE                                                     [1, 3, 64, 64]            1
├─SwinTransformer: 1-1                                  [1, 512]                  --
│    └─Sequential: 2-1                                  [1, 8, 8, 192]            --
│    │    └─Sequential: 3-1                             [1, 16, 16, 96]           4,896
│    │    └─Sequential: 3-2                             [1, 16, 16, 96]           455,472
│    │    └─PatchMerging: 3-3                           [1, 8, 8, 192]            74,496
│    │    └─Sequential: 3-4                             [1, 8, 8, 192]            2,693,520
│    └─LayerNorm: 2-2                                   [1, 8, 8, 192]            384
│    └─Permute: 2-3                                     [1, 192, 8, 8]            --
│    └─AdaptiveAvgPool2d: 2-4                           [1, 192, 1, 1]            --
│    └─Flatt

/home/uz1/miniconda3/envs/ML/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
/home/uz1/miniconda3/envs/ML/lib/python3.7/site-packages/lightning_lite/plugins/environments/slurm.py:172: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/uz1/miniconda3/envs/ML/lib/python3.7/site-pack ...
  category=PossibleUserWarning,
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


/applications/slurm-22.05.2/bin/srun


...

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/home/uz1/miniconda3/envs/ML/lib/python3.7/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [9]:
# test results of recondtruction
model.eval()
import random
i= random.randint(0,len(data))
with torch.no_grad():
    x = data[i]
    x_hat = model(x)
    x_hat = x_hat.squeeze(0)
    x_hat = x_hat.cpu().numpy()
    x = x[0].cpu().numpy()
    x_hat = np.moveaxis(x_hat, 1, -1)
    x = np.moveaxis(x, 1, -1)
    print(x_hat.shape)

# plot - handle multiple images , x is multiple images
# for tht length of x.shape[0] 
for i in range(x.shape[0]):
    plt.figure(figsize=(10,10))
    plt.subplot(1,2,1)
    plt.imshow(x[i])
    plt.subplot(1,2,2)
    plt.imshow(x_hat[i])
    plt.show()


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_2926287/3404544689.py:7 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_2926287/3404544689.py'                      │
│                                                                                                  │
│ /home/uz1/miniconda3/envs/ML/lib/python3.7/site-packages/torch/nn/modules/module.py:1194 in      │
│ _call_impl                                                                                       │
│                                                                                                  │
│   1191 │   │   # this function, and just call forward.                                           │
│   1192 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1193 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1194 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1195 │   │   # Do not call functions when jit is used                                          │
│   1196 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1197 │   │   if self._backward_hooks or _global_backward_hooks:                                │
│                                                                                                  │
│ /tmp/ipykernel_2926287/2457605489.py:122 in forward                                              │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_2926287/2457605489.py'                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'tuple' object has no attribute 'dim'

In [12]:
#using mini batch kemeans 
from sklearn.cluster import MiniBatchKMeans
import math
import pickle
from math import sqrt
from tqdm import tqdm
data,loader= get_data(pz = pz,img_size= 224,pz_=None,split='val')
num_nodes = 168
num_points = 700 if len(data) > 20000 else len(data)
model2 = VAE(input_height=inp,window_size =28, patch_size = 4)
vae = model2.load_from_checkpoint("/home/uz1/projects/GCN/logging/swin_vae/2023_03_16/epoch=0-step=8999.ckpt",) # 64
h=int(int(data[0][0].shape[-1]) * sqrt(data[0][0].shape[0]))
p_z = int(sqrt((h*h) // int(data[0][0].shape[0])))
num_points = num_points if (((h // p_z) * (h // p_z) ) * num_points ) % 16000 == 0 else 16000 // ((h // p_z) * (h // p_z))
num_patches= (h // p_z) * (h // p_z)
num_points = 16000 // (num_patches * loader.batch_size)
print("given num_patches: ", num_patches, " num_points: ", num_points)

vae = vae.to('cpu')




h = 224
p_z = data[0][0].shape[-1]
# print("Using a VAE with h=",h,"and p(z)=",p_z)
if loader.batch_size < num_nodes:
    batch_size = num_nodes
    loader = DataLoader(data, batch_size=batch_size, drop_last=True, num_workers=14)
if not os.path.exists(f"/home/uz1/projects/GCN/GraphGym/run/kmeans-model-{h}-{p_z}-{num_nodes}-{data.__class__.__name__}-swin.pkl"):
    kmeans = MiniBatchKMeans(n_clusters=num_nodes)
    for i in tqdm(range(num_points),total=num_points,desc="Fitting Kmeans",):
        test,_ = next(iter(loader))
        test = test.to(vae.device)
        if test.dim() >4:
            test = test.reshape(-1, test.shape[2], test.shape[3], test.shape[4]) # batch, num patches ,channel, height, width to batch, channel, height, width
        x_encoded = vae.encoder(test)
        mu, log_var = vae.fc_mu(x_encoded), vae.fc_var(x_encoded)
        std = torch.exp(log_var / 2)
        q = torch.distributions.Normal(mu, std)
        z = q.rsample()
        z=z.detach().cpu().numpy()
        kmeans.partial_fit(z)
    print("Done - out shape ",z.shape)
    with open(f"/home/uz1/projects/GCN/GraphGym/run/kmeans-model-{h}-{p_z}-{num_nodes}-{data.__class__.__name__}-swin.pkl", 'wb') as f:
        pickle.dump(kmeans, f)

print("Kmeans Done !")



config:
 patch_size: 64 img_size: 224 num_patches: 9 bs: 10


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_1624640/164525078.py:10 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_1624640/164525078.py'                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'inp' is not defined

In [13]:
pz=64
data,loader= get_data(pz = pz,img_size= 224,pz_=None,split='train')
num_nodes = 168
num_points = 700 if len(data) > 20000 else len(data)
model2 = VAE(input_height=224,window_size =28, patch_size = 4)
vae = model2.load_from_checkpoint("/home/uz1/projects/GCN/logging/swin_vae/2023_03_16/epoch=0-step=8999.ckpt",) # 64

config:
 patch_size: 64 img_size: 224 num_patches: 9 bs: 10
using input height  224  and latent dim  256  and enc_out_dim  512


/home/uz1/miniconda3/envs/ML/lib/python3.7/site-packages/ipykernel_launcher.py:81: UnderReviewWarning: The feature DecoderBlock is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html


using input height  64  and latent dim  256  and enc_out_dim  512


In [33]:

# %%
def get_embedding_vae(x,vae):

    x_encoded = vae.encoder(x)
    mu, log_var = vae.fc_mu(x_encoded), vae.fc_var(x_encoded)
    std = torch.exp(log_var / 2)
    q = torch.distributions.Normal(mu, std)
    z = q.rsample()
    return z


from torch_geometric.data import Data,Dataset
from torch_geometric.utils import to_dense_adj, grid,dense_to_sparse

def filter_a(data):

    if data.y==3:
        return False
    else:
        True
def populateSBatched(labels, n_clusters=8, s=None, n_graphs=10, n_patches=16):
    """
    Calculates the S cluster assignment transform of input patch features 
    for multiple graphs and returns the S and the aggregated adjacency matrix for each graph.
    Shape : (number of patches, number of clusters)

    Args:
        labels (ndarray): Labels for each node in each graph. Shape is (n_graphs*n_patches,)
        n_clusters (int): Number of clusters
        s (ndarray): Existing S assignment transform to update
        n_graphs (int): Number of graphs
        n_patches (int): Number of patches in each image

    Returns:
        tuple: S assignment transform and aggregated adjacency matrix for each graph
    """
    # n_patches = n_graphs * n_patches
    div = int(np.sqrt(n_patches))

    S_list = []
    adj_list = []

    for i in range(n_graphs):
        start = i * n_patches
        end = start + n_patches
        graph_labels = labels[start:end]

        s_graph = np.zeros((n_patches, n_clusters))
        s_graph[np.arange(n_patches), graph_labels] = 1

        # calc adj matrix
        adj_graph = to_dense_adj(grid(div, div)[0]).reshape(n_patches, n_patches)

        S_list.append(s_graph)
        adj_list.append(np.matmul(np.matmul(s_graph.transpose(1, 0), adj_graph), s_graph))

    S = np.concatenate(S_list)
    adj = np.stack(adj_list)

    return S, adj

def populateS(labels,n_clusters=8,s=None):
    """"
    Calculates the S cluster assigment transform of input patch features 
    and returns the (S) and the aggregated (out_adj) as well.
    shape : ( number of patches , number of clusters)
    """
    # print("S is " ,s==None)
    n_patches=len(labels)
    div = int(sqrt(n_patches))
 
    s = np.zeros((n_patches,n_clusters))
    s[np.arange(n_patches), labels] = 1
         # TODO optimise this!


    #calc adj matrix
    adj = to_dense_adj(grid(n_patches//div,n_patches//div)[0]).reshape(n_patches,n_patches)
    return s , np.matmul(np.matmul(s.transpose(1, 0),adj ), s)

from torch_geometric.data import Data,Dataset
from torch_geometric.utils import to_dense_adj, grid,dense_to_sparse

from monai.data import GridPatchDataset, DataLoader, PatchIter


class ImageTOGraphDatasetBatched(Dataset):
    """ 
    Dataset takes holds the kmaens classifier and vae encoder. On each input image we encode then get k mean label then formulate graph as Data object
    """
    def __init__(self,data,vae,kmeans,norm_adj=True,return_x_only=None):
        self.data=data
        self.vae=vae
        self.vae.cuda()
        self.return_x_only=return_x_only

        self.kmeans = kmeans
        self.norm_adj = norm_adj
        self.n_patches = self.data[1][0].shape[0]

        # if self.data[1][0].dim() >3:
        #     self.__getitem__ = self.__getitem__patches   
            # add attr

    def __len__(self):
        return len(self.data)
    def __getitem__(self,index):
        #given that index is a list of indeices , load the patches and get the embedding
        # p = self.data[index][0] wont work 
    
        b=len(index) 
        patches = torch.concat([self.data[i][0] for i in index],0).cuda()
        n_graphs = patches.shape[0] // self.n_patches
        n_clusters = self.kmeans.cluster_centers_.shape[0]
        ys = np.concatenate([self.data[i][1] for i in index],0)
        z=get_embedding_vae(patches,self.vae.cuda()).clone().detach().cpu().numpy()
        label=self.kmeans.predict(z)
        s,out_adj = populateSBatched(label,n_clusters,n_graphs=n_graphs,n_patches=self.n_patches)

        z = z.reshape((b, self.n_patches, -1))
        s = s.reshape((b,self.n_patches, n_clusters))

        s = np.transpose(s,(0,2,1))
        x = np.matmul(s, z).reshape(b,n_clusters,-1)
        del z,s,patches
        if self.return_x_only:
            return torch.tensor(x),torch.tensor(label),torch.tensor(ys)
        #if normlaise adj 
        if self.norm_adj:
            out_adj = out_adj.div(out_adj.sum(1))
            #nan to 0 in tensor 
            out_adj = out_adj.nan_to_num(0)
            #assert if there is nan in tensor
            assert out_adj.isnan().any() == False , "Found nan in out_adj"
        return Data(x=torch.tensor(x).float(),edge_index=dense_to_sparse(out_adj)[0],y=torch.tensor(self.data[index][1]),edge_attr=dense_to_sparse(out_adj)[1])
class ImageTOGraphDataset(Dataset):
    """ 
    Dataset takes holds the kmaens classifier and vae encoder. On each input image we encode then get k mean label then formulate graph as Data object
    """
    def __init__(self,data,vae,kmeans,norm_adj=True,return_x_only=None):
        self.data=data
        self.vae=vae
        self.return_x_only=return_x_only

        self.kmeans = kmeans
        self.norm_adj = norm_adj

        # if self.data[1][0].dim() >3:
        #     self.__getitem__ = self.__getitem__patches   
            # add attr

    def __len__(self):
        return len(self.data)
    def __getitem__(self,index):

        patches = self.data[index][0]
        z=get_embedding_vae(patches.cuda(),self.vae.cuda()).clone().detach().cpu().numpy()
        label=self.kmeans.predict(z)
        s,out_adj = populateS(label,self.kmeans.cluster_centers_.shape[0])
        x = np.matmul(s.transpose(1,0) , z)
        del z
        del s,patches
        if self.return_x_only:
            return torch.tensor(x),torch.tensor(label),torch.tensor(self.data[index][1])
        #if normlaise adj 
        if self.norm_adj:
            out_adj = out_adj.div(out_adj.sum(1))
            #nan to 0 in tensor 
            out_adj = out_adj.nan_to_num(0)
            #assert if there is nan in tensor
            assert out_adj.isnan().any() == False , "Found nan in out_adj"
        return Data(x=torch.tensor(x).float(),edge_index=dense_to_sparse(out_adj)[0],y=torch.tensor(self.data[index][1]),edge_attr=dense_to_sparse(out_adj)[1])


In [15]:
# load /home/uz1/projects/GCN/GraphGym/run/kmeans-model-{h}-{p_z}-{num_nodes}-{data.__class__.__name__}-swin.pkl
h=224
import pickle
with open(f"/home/uz1/projects/GCN/GraphGym/run/kmeans-model-{h}-{pz}-{num_nodes}-{data.__class__.__name__}-swin.pkl","rb") as f:
    k = pickle.load(f)
ImData = ImageTOGraphDatasetBatched(data=data,vae=vae,kmeans=k,return_x_only=True)
sampler = torch.utils.data.sampler.BatchSampler(
    torch.utils.data.sampler.RandomSampler(ImData),
    batch_size=loader.batch_size,
    drop_last=False)

In [27]:
h5f.close()

In [53]:
from torch.utils.data import DataLoader
# train_loader = DataLoader(ImData, batch_size=args.batch_size,)
train_loader = DataLoader(ImData, batch_size=1,sampler = sampler)
# for all data in the dataset: create a h5y file to store the data and save to disk
import h5py
import os
import numpy as np
from torch_geometric.data import Data,Dataset
from tqdm import tqdm
#create h5py file
# if os.path.exists(f'/home/uz1/projects/GCN/graph_data/graph-data---{args.dataset}-{args.patch_size}-{args.nclusters}-{args.image_transform_size}-UC_{args.use_combined}.h5'):
    # print("File already exists at ", f'/home/uz1/projects/GCN/graph_data/graph-data---{args.dataset}-{args.patch_size}-{args.nclusters}-{args.image_transform_size}-UC_{args.use_combined}.h5')

    #exit
    # exit("File already exists")

h5f = h5py.File(f'/home/uz1/projects/GCN/graph_data/graph-data---pathmnist-{pz}-{num_nodes}-{h}-swin_.h5', 'w')

#create dataset in file

h5f.create_dataset('x', (len(ImData),num_nodes,vae.fc_mu.out_features))

# add the data to h5py file
n_clusters = num_nodes
embed_size = vae.fc_mu.out_features
n_patches = data[0][0].shape[0]
print(f"n_clusters: {n_clusters}, embed_size: {embed_size}, n_patches: {n_patches}")
#try numpy array
edge_i = []
edge_at = []
ys=[]
tmp=0
for i,d in tqdm(enumerate(train_loader),total=len(train_loader)):

    h5f['x'][tmp:tmp+d[0].squeeze().shape[0]] = d[0].reshape(-1,n_clusters,embed_size)
    tmp=tmp+d[0].squeeze().shape[0]

    ys.append(d[2].squeeze())
    edge_i.append(d[1].squeeze())

edge_i = np.stack(edge_i[:-1])
edge_i = edge_i.reshape(-1,ImData[[0]][1].shape[0])
ys = np.stack(ys[:-1])
ys = np.concatenate([ys.reshape(-1),d[2].reshape(-1)]) 
edge_i = np.concatenate([edge_i.reshape(-1,n_patches),d[1].reshape(-1,n_patches)])

#create datasets for edge index and edge attr and ys
h5f.create_dataset('edge_index', data=edge_i)
h5f.create_dataset('ys', data=ys)

#close the file
h5f.close()

n_clusters: 168, embed_size: 256, n_patches: 9


100%|██████████| 9000/9000 [1:01:38<00:00,  2.43it/s]


In [47]:
edge_i.reshape(-1,ImData[[0]][1].shape[0])

(89996, 9)

In [51]:
np.concatenate([edge_i.reshape(-1,n_patches),d[1].reshape(-1,n_patches)]).shape

(270024, 3)

In [52]:
data[0][0].shape

torch.Size([9, 3, 64, 64])